In [10]:
from typing import Literal, Annotated, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.graph import END

from dotenv import load_dotenv
import os

from langchain.agents import Tool
from langchain.tools import StructuredTool
from notion.planner import Planner, ConfirmablePlanner
from notion.task import Task
from pydantic import BaseModel, Field

from datetime import datetime, timedelta
import json

load_dotenv()

True

In [45]:
# 기본 타입 베이스
class DateSchema(BaseModel):
    start_datetime: Annotated[str, "Start datetime string of the task, format: 'YYYY-MM-DDTHH:MM'"]
    end_datetime: Annotated[str, "End datetime string of the task, format: 'YYYY-MM-DDTHH:MM'"]

class TaskSchema(BaseModel):
    name: Annotated[str, "A new task name to create"]
    date: Annotated[DateSchema, "A new task date to create. if there are single datetime, make end_datetime as same as start_datetime"]
    group: Annotated[str, "A new task group name to create"]


class CreateTaskSchema(BaseModel):
    task: Annotated[TaskSchema, "A new task to create"]
    
def create_task(task):
    print(task)
    task.date.start_datetime = datetime.strptime(task.date.start_datetime, "%Y-%m-%dT%H:%M")
    task.date.end_datetime = datetime.strptime(task.date.end_datetime, "%Y-%m-%dT%H:%M")
    try:
        task = Task(
            name=task.name,
            date=task.date,
            group=task.group
        )
    except Exception as e:
        return f"Error occured with creating task: {str(e)}, stop creating task"

    return "Successfully created task: " + task.name

tools = [
    StructuredTool.from_function(
        func=create_task,
        name="create_task",
        description="Create a new task.",
        args_schema=CreateTaskSchema
    ), 
]

tool_agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Your task is calling a tool as user requested. "
            "Answer in Korean.",
        ),
        ("human", "{messages}"),
    ]
)


from langgraph.prebuilt import create_react_agent

# 대화 및 요약을 위한 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

agent_executor = create_react_agent(
    model=model,
    tools=tools,
    prompt=tool_agent_prompt
)

In [46]:
query = "'테스트'라는 이름의 일정을 '이벤트' 그룹에 추가하고, 시작일은 2023년 10월 1일 오후 2시, 종료일은 2023년 10월 1일 오후 4시로 설정해줘."

result = agent_executor.invoke({"messages": query})
print(result)

name='테스트' date=DateSchema(start_datetime='2023-10-01T14:00:00', end_datetime='2023-10-01T16:00:00') group='이벤트'
name='테스트' date=DateSchema(start_datetime='2023-10-01T14:00:00', end_datetime='2023-10-01T16:00:00') group='이벤트'
name='테스트' date=DateSchema(start_datetime='2023-10-01T14:00:00', end_datetime='2023-10-01T16:00:00') group='이벤트'
name='테스트' date=DateSchema(start_datetime='2023-10-01T14:00:00', end_datetime='2023-10-01T16:00:00') group='이벤트'


KeyboardInterrupt: 

In [41]:
from pprint import pprint
# 결과 출력
pprint(result)

{'messages': [HumanMessage(content="'테스트'라는 이름의 일정을 '이벤트' 그룹에 추가하고, 시작일은 2023년 10월 1일 오후 2시, 종료일은 2023년 10월 1일 오후 4시로 설정해줘.", additional_kwargs={}, response_metadata={}, id='a9b8362d-adf7-458e-8eb5-46c5f419dc78'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pOotC4NL8U6H5y0Ci9iQvTXo', 'function': {'arguments': '{"task":{"name":"테스트","date":{"start_datetime":"2023-10-01T14:00:00","end_datetime":"2023-10-01T16:00:00"},"group":"이벤트"}}', 'name': 'create_task'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 178, 'total_tokens': 235, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e6586cb4-a0e6

In [38]:
messages = result['messages']
for message in messages:
    if isinstance(message, HumanMessage):
        print("User: ", message.content)
    elif isinstance(message, SystemMessage):
        print("System: ", message.content)
    elif isinstance(message, AIMessage):
        ai_content = message.content
        if ai_content == "":
            tool_args = message.additional_kwargs['tool_calls'][0]['function']['arguments']
            print("Tool call: ", tool_args)
        else:
            print("AI: ", message.content)
    elif isinstance(message, ToolMessage):
        print("Tool: ", message.content)
    else:
        print("Unknown message type:", type(message))

User:  '테스트'라는 이름의 일정을 '이벤트' 그룹에 추가하고, 시작일은 2023년 10월 1일 오후 2시, 종료일은 2023년 10월 1일 오후 4시로 설정해줘.
Tool call:  {"task":{"name":"테스트","date":{"start_datetime":"2023-10-01T14:00:00","end_datetime":"2023-10-01T16:00:00"},"group":"이벤트"}}
Tool:  Successfully created task: 테스트
AI:  일정 '테스트'가 '이벤트' 그룹에 성공적으로 추가되었습니다. 시작일은 2023년 10월 1일 오후 2시, 종료일은 2023년 10월 1일 오후 4시입니다.
